# Linear Regression: The Closed-Form Solution

This notebook demonstrates the ordinary least squares (OLS) closed-form solution for linear regression, connecting the matrix formulation in Chapter 2 to concrete computation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mlone_theme as mt

mt.set_notebook_mode()
np.set_printoptions(precision=4, suppress=True)

## 1. The Normal Equations

For a linear model $\mathbf{y} = \mathbf{X}\boldsymbol{\beta} + \boldsymbol{\varepsilon}$, the OLS estimator minimizes the residual sum of squares:

$$\hat{\boldsymbol{\beta}} = \arg\min_{\boldsymbol{\beta}} \|\mathbf{y} - \mathbf{X}\boldsymbol{\beta}\|^2$$

Taking the derivative and setting it to zero yields the **normal equations**:

$$\mathbf{X}^\top \mathbf{X} \hat{\boldsymbol{\beta}} = \mathbf{X}^\top \mathbf{y}$$

When $\mathbf{X}^\top \mathbf{X}$ is invertible, we get the closed-form solution:

$$\hat{\boldsymbol{\beta}} = (\mathbf{X}^\top \mathbf{X})^{-1} \mathbf{X}^\top \mathbf{y}$$

In [ ]:
def ols_closed_form(X, y):
    """Compute OLS coefficients using the normal equations."""
    XtX = X.T @ X
    Xty = X.T @ y
    beta = np.linalg.inv(XtX) @ Xty
    return beta

## 2. A Small Worked Example

Consider a simple dataset with 5 observations. We'll compute everything step-by-step to see the matrix operations in action.

In [ ]:
# Simple dataset: house size (hundreds of sq ft) vs price (thousands of $)
x = np.array([8, 10, 12, 14, 16])     # Feature
y = np.array([150, 200, 250, 280, 310])  # Target

print("Feature (x):  ", x)
print("Target  (y):  ", y)

In [ ]:
# Build the design matrix with intercept column
X = np.column_stack([np.ones(len(x)), x])
print("Design matrix X:")
print(X)

The design matrix has a column of ones for the intercept term. Now let's compute each component of the normal equations.

In [ ]:
# Step 1: Compute X^T X
XtX = X.T @ X
print("X^T X:")
print(XtX)

In [ ]:
# Step 2: Compute X^T y
Xty = X.T @ y
print("X^T y:")
print(Xty)

In [ ]:
# Step 3: Compute (X^T X)^{-1}
XtX_inv = np.linalg.inv(XtX)
print("(X^T X)^{-1}:")
print(XtX_inv)

In [ ]:
# Step 4: Compute beta = (X^T X)^{-1} X^T y
beta = XtX_inv @ Xty
print(f"Estimated coefficients: beta_0 = {beta[0]:.2f}, beta_1 = {beta[1]:.2f}")
print(f"\nInterpretation: Price = {beta[0]:.0f} + {beta[1]:.1f} × Size")

## 3. Visualization

In [ ]:
# Predictions
y_pred = X @ beta
residuals = y - y_pred

print("Actual:     ", y)
print("Predicted:  ", y_pred.round(1))
print("Residuals:  ", residuals.round(1))

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))

# Data points
mt.scatter_with_border(ax, x, y, color=mt.GREEN, size=80)

# Regression line
x_line = np.linspace(6, 18, 100)
y_line = beta[0] + beta[1] * x_line
ax.plot(x_line, y_line, color=mt.GREEN, linewidth=2, label='OLS fit')

# Show residuals as vertical lines
for xi, yi, ypi in zip(x, y, y_pred):
    ax.plot([xi, xi], [yi, ypi], color=mt.RED, linewidth=1.5, linestyle='--', alpha=0.7)

ax.set_xlabel('House Size (hundreds of sq ft)')
ax.set_ylabel('Price (thousands of $)')
mt.apply_economist_style(ax)
plt.tight_layout()
plt.show()

The dashed red lines show the residuals—the vertical distances that OLS minimizes in aggregate.

## 4. The Geometric View: Projection

The OLS solution projects the target vector $\mathbf{y}$ onto the column space of $\mathbf{X}$. The fitted values $\hat{\mathbf{y}} = \mathbf{X}\hat{\boldsymbol{\beta}}$ are the closest point to $\mathbf{y}$ in this subspace.

The **hat matrix** $\mathbf{H} = \mathbf{X}(\mathbf{X}^\top\mathbf{X})^{-1}\mathbf{X}^\top$ performs this projection:

In [ ]:
# Hat matrix
H = X @ np.linalg.inv(X.T @ X) @ X.T
print("Hat matrix H (5×5):")
print(H.round(3))

In [ ]:
# Verify: H @ y gives fitted values
y_hat_via_H = H @ y
print("Fitted values via H @ y:  ", y_hat_via_H.round(1))
print("Fitted values via X @ β:  ", y_pred.round(1))

## 5. Connecting to scikit-learn

Of course, in practice we use library implementations. Let's verify that sklearn's `LinearRegression` gives the same result.

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x.reshape(-1, 1), y)  # sklearn expects 2D input

print(f"sklearn intercept: {model.intercept_:.2f}")
print(f"sklearn slope:     {model.coef_[0]:.2f}")
print(f"\nOur intercept:     {beta[0]:.2f}")
print(f"Our slope:         {beta[1]:.2f}")

## 6. Multiple Regression

The same formulation extends to multiple features. Let's add a second feature: number of bedrooms.

In [ ]:
# Extended dataset with two features
np.random.seed(42)
n = 50

size = np.random.uniform(8, 20, n)       # House size
bedrooms = np.random.randint(2, 5, n)    # Number of bedrooms

# True relationship: Price = 20 + 15*size + 25*bedrooms + noise
price = 20 + 15 * size + 25 * bedrooms + np.random.normal(0, 15, n)

print(f"Generated {n} observations with 2 features")

In [ ]:
# Design matrix with intercept
X_multi = np.column_stack([np.ones(n), size, bedrooms])
print(f"Design matrix shape: {X_multi.shape}")
print("\nFirst 5 rows:")
print(X_multi[:5])

In [ ]:
# OLS solution
beta_multi = ols_closed_form(X_multi, price)

print("Estimated coefficients:")
print(f"  Intercept:       {beta_multi[0]:.2f} (true: 20)")
print(f"  Size effect:     {beta_multi[1]:.2f} (true: 15)")
print(f"  Bedroom effect:  {beta_multi[2]:.2f} (true: 25)")

In [ ]:
# Verify with sklearn
X_sklearn = np.column_stack([size, bedrooms])
model_multi = LinearRegression().fit(X_sklearn, price)

print("\nsklearn coefficients:")
print(f"  Intercept:       {model_multi.intercept_:.2f}")
print(f"  Size effect:     {model_multi.coef_[0]:.2f}")
print(f"  Bedroom effect:  {model_multi.coef_[1]:.2f}")

## 7. Residual Analysis

Examining residuals helps verify model assumptions: they should be roughly normally distributed with constant variance.

In [ ]:
# Compute residuals for multiple regression
y_pred_multi = X_multi @ beta_multi
residuals_multi = price - y_pred_multi

fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# 1. Predicted vs Actual
ax = axes[0]
mt.scatter_with_border(ax, y_pred_multi, price, color=mt.GREEN, size=40)
ax.plot([100, 350], [100, 350], 'k--', alpha=0.5, linewidth=1)
ax.set_xlabel('Predicted Price')
ax.set_ylabel('Actual Price')
mt.apply_economist_style(ax)
ax.set_title('Predicted vs Actual', fontsize=12, loc='left')

# 2. Residuals vs Fitted
ax = axes[1]
mt.scatter_with_border(ax, y_pred_multi, residuals_multi, color=mt.GREEN, size=40)
ax.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax.set_xlabel('Fitted Values')
ax.set_ylabel('Residuals')
mt.apply_economist_style(ax)
ax.set_title('Residuals vs Fitted', fontsize=12, loc='left')

# 3. Histogram of residuals
ax = axes[2]
ax.hist(residuals_multi, bins=15, color=mt.GREEN, edgecolor='white', alpha=0.8)
ax.set_xlabel('Residual')
ax.set_ylabel('Count')
mt.apply_economist_style(ax)
ax.set_title('Residual Distribution', fontsize=12, loc='left')

plt.tight_layout()
plt.show()

The residuals show no obvious pattern against fitted values (good—constant variance) and are roughly symmetric around zero (good—no systematic bias).

## 8. Numerical Stability: Using `lstsq`

Computing $(\mathbf{X}^\top\mathbf{X})^{-1}$ directly can be numerically unstable for ill-conditioned matrices. NumPy's `lstsq` uses the more stable SVD-based approach:

In [ ]:
# Using numpy's lstsq (more numerically stable)
beta_lstsq, residuals_lstsq, rank, singular_values = np.linalg.lstsq(X_multi, price, rcond=None)

print("Coefficients via lstsq:")
print(f"  Intercept:       {beta_lstsq[0]:.4f}")
print(f"  Size effect:     {beta_lstsq[1]:.4f}")
print(f"  Bedroom effect:  {beta_lstsq[2]:.4f}")

print(f"\nDifference from closed-form: {np.max(np.abs(beta_lstsq - beta_multi)):.2e}")

## Summary

The closed-form OLS solution $\hat{\boldsymbol{\beta}} = (\mathbf{X}^\top\mathbf{X})^{-1}\mathbf{X}^\top\mathbf{y}$ provides:

- **Direct computation** of optimal coefficients (no iteration)
- **Geometric interpretation** as projection onto column space of $\mathbf{X}$
- **Exact solution** (up to numerical precision)

However, the closed-form approach has limitations:
- Requires $\mathbf{X}^\top\mathbf{X}$ to be invertible
- Computationally expensive for large $n$ (matrix inversion is $O(p^3)$)
- Cannot easily incorporate regularization

These limitations motivate **gradient descent**, which we explore in the next notebook.